In [6]:
import pandas as pd

Fichier Discounts.csv

In [7]:
data = pd.read_csv('Old_Data/discounts.csv', sep=",")
data.head()

,Start,End,Discont,Description,Category,Sub Category
0,2020-01-01,2020-01-10,0.4,40% discount during our New Year Winter Sale,Feminine,Coats and Blazers
1,2020-01-01,2020-01-10,0.4,40% discount during our New Year Winter Sale,Feminine,Sweaters and Knitwear
2,2020-01-01,2020-01-10,0.4,40% discount during our New Year Winter Sale,Masculine,Coats and Blazers
3,2020-01-01,2020-01-10,0.4,40% discount during our New Year Winter Sale,Masculine,Sweaters and Sweatshirts
4,2020-01-01,2020-01-10,0.4,40% discount during our New Year Winter Sale,Children,Coats


In [8]:
df_Discounts = data[['Start', 'End', 'Discont']]    

In [9]:
df_Discounts.head()

,Start,End,Discont
0,2020-01-01,2020-01-10,0.4
1,2020-01-01,2020-01-10,0.4
2,2020-01-01,2020-01-10,0.4
3,2020-01-01,2020-01-10,0.4
4,2020-01-01,2020-01-10,0.4


In [10]:
df_Discounts.to_csv("Data/discounts.csv", index=False)

Fichier Stores.csv

In [11]:
data = pd.read_csv('Old_Data/stores.csv', sep=",")
data.head()

,Store ID,Country,City,Store Name,Number of Employees,ZIP Code,Latitude,Longitude
0,1,United States,New York,Store New York,10,10001,40.7128,-74.0060
1,2,United States,Los Angeles,Store Los Angeles,8,90001,34.0522,-118.2437
2,3,United States,Chicago,Store Chicago,9,60601,41.8781,-87.6298
3,4,United States,Houston,Store Houston,10,77001,29.7604,-95.3698
4,5,United States,Phoenix,Store Phoenix,9,85001,33.4484,-112.0740


In [12]:
df_Stores = data[['Store ID', 'Store Name', 'City', 'Country', 'ZIP Code', 'Latitude', 'Longitude']]

In [13]:
df_Stores.head()

,Store ID,Store Name,City,Country,ZIP Code,Latitude,Longitude
0,1,Store New York,New York,United States,10001,40.7128,-74.0060
1,2,Store Los Angeles,Los Angeles,United States,90001,34.0522,-118.2437
2,3,Store Chicago,Chicago,United States,60601,41.8781,-87.6298
3,4,Store Houston,Houston,United States,77001,29.7604,-95.3698
4,5,Store Phoenix,Phoenix,United States,85001,33.4484,-112.0740


In [ ]:
df_Stores.to_csv("Data/stores.csv", index=False)

OSError: Cannot save file into a non-existent directory: 'Data\Data'

Fichier Products

In [ ]:
data = pd.read_csv('Old_Data/products.csv', sep=",")
data.head()

,Product ID,Category,Sub Category,Description PT,Description DE,Description FR,Description ES,Description EN,Description ZH,Color,Sizes,Production Cost
0,1,Feminine,Coats and Blazers,Esportivo Veludo Verde Com Botões,Sport Samt Sport Mit Knöpfen,Sports Velvet Sports Avec Des Boutons,Deportes De Terciopelo Con Botones,Sports Velvet Sports With Buttons,运动天鹅绒运动与按钮,NaN,S|M|L|XL,10.73
1,2,Feminine,Sweaters and Knitwear,Luxuoso Denim Rosa Com Botões,Luxuriöser Rosa Jeans Mit Knöpfen,Léchard De Denim Rose Avec Boutons,Denim Rosa Lujoso Con Botones,Luxurious Pink Denim With Buttons,豪华的粉红色牛仔布和纽扣,PINK,S|M|L|XL,19.55
2,3,Feminine,Dresses and Jumpsuits,Retrô Tricot Preto Estampado,Black Tricot Gedruckter Tricot,Tricot Imprimé En Tricot Noir,Tricot Negro Tricot Impreso,Black Tricot Printed Tricot,黑色三角形印刷三角形,BLACK,S|M|L|XL,25.59
3,4,Feminine,Shirts and Blouses,Blusa De Algodão Básica,Basis -Baumwollbluse,Chemisier En Coton De Base,Blusa De Algodón,Basic Cotton Blouse,基本的棉衬衫,NaN,S|M|L|XL,27.62
4,5,Feminine,T-shirts and Tops,T-Shirt Básica De Algodão,Basis-Baumwoll-T-Shirt,T-Shirt En Coton De Base,Camiseta Básica De Algodón,Basic Cotton T-Shirt,基本棉T恤,NaN,S|M|L,11.69


In [ ]:
df_Products = data[['Product ID','Category', 'Sub Category', 'Description FR', 'Sizes', 'Production Cost']]    

In [ ]:
df_Products.head()

,Product ID,Category,Sub Category,Description FR,Sizes,Production Cost
0,1,Feminine,Coats and Blazers,Sports Velvet Sports Avec Des Boutons,S|M|L|XL,10.73
1,2,Feminine,Sweaters and Knitwear,Léchard De Denim Rose Avec Boutons,S|M|L|XL,19.55
2,3,Feminine,Dresses and Jumpsuits,Tricot Imprimé En Tricot Noir,S|M|L|XL,25.59
3,4,Feminine,Shirts and Blouses,Chemisier En Coton De Base,S|M|L|XL,27.62
4,5,Feminine,T-shirts and Tops,T-Shirt En Coton De Base,S|M|L,11.69


In [ ]:
df_Products.to_csv("Data/products.csv", index=False)

Fichier Transactions
- Création des fichiers date.csv et transactions.csv

In [ ]:
data = pd.read_csv('Old_Data/transactions.csv', sep=",")
data.head()

In [ ]:
df_Date = data[['Date']] 
df_Date = df_Date.drop_duplicates()
df_Date = df_Date.reset_index(drop=True)
df_Date['Date Key'] = df_Date.index +1

In [ ]:
df_Date.head()

,Date,Date Key
0,2023-01-01 15:42:00,1
1,2023-01-01 20:04:00,2
2,2023-01-01 20:47:00,3
3,2023-01-01 08:06:00,4
4,2023-01-01 17:25:00,5


In [ ]:
df_Date.to_csv("Data/date.csv", index=False)

In [ ]:
df_Transactions = data[['Product ID', 'Store ID', 'Quantity', 'Unit Price', 'Line Total', 'Transaction Type', 'Invoice Total']]    
df_Transactions = df_Transactions.drop_duplicates()
df_Transactions = df_Transactions.reset_index(drop=True)
df_Transactions['Transactions Key'] = df_Transactions.index +1

In [ ]:
df_Transactions.head()

,Product ID,Store ID,Quantity,Unit Price,Line Total,Transaction Type,Invoice Total,Transactions Key
0,485,1,1,80.5,80.5,Sale,126.7,1
1,2779,1,1,31.5,18.9,Sale,126.7,2
2,64,1,1,45.5,27.3,Sale,126.7,3
3,131,1,1,70.0,42.0,Sale,77.0,4
4,716,1,1,26.0,26.0,Sale,77.0,5


In [ ]:
df_Date.to_csv("Data/transactions.csv", index=False)